In [16]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M, B
from chip_estimates_utils import (
    normalize_shares,
    compute_h100_equivalents,
    export_quarterly_by_version,
    print_cumulative_summary,
    estimate_chip_sales
)

sq.set_seed(42)
np.random.seed(42)
N_SAMPLES = 5000
H100_TOPS = 1979

In [17]:
# ==============================================
# GOOGLE SHEETS CONFIGURATION
# ==============================================
# Spreadsheet: https://docs.google.com/spreadsheets/d/1RfjL98gsSe9QeXDnBOraticvS-s13XmKvHv_dwwa3W8/

SPREADSHEET_ID = "1RfjL98gsSe9QeXDnBOraticvS-s13XmKvHv_dwwa3W8"

# Sheet GIDs
REVENUE_GID = 0
PROD_MIX_GID = 141507805

# Construct URLs for direct CSV export using GID
REVENUE_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&gid={REVENUE_GID}"
PROD_MIX_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&gid={PROD_MIX_GID}"
print(REVENUE_URL)
print(PROD_MIX_URL)

https://docs.google.com/spreadsheets/d/1RfjL98gsSe9QeXDnBOraticvS-s13XmKvHv_dwwa3W8/gviz/tq?tqx=out:csv&gid=0
https://docs.google.com/spreadsheets/d/1RfjL98gsSe9QeXDnBOraticvS-s13XmKvHv_dwwa3W8/gviz/tq?tqx=out:csv&gid=141507805


In [18]:
# ==============================================
# LOAD DATA FROM GOOGLE SHEETS
# ==============================================

print("Loading AMD Instinct revenue data from Google Sheets...")
revenue_df = pd.read_csv(REVENUE_URL)
# Clean up any extra columns - quarter format is "FY24 Q1"
revenue_df = revenue_df[['quarter', 'instinct_revenue_p5', 'instinct_revenue_p95']].dropna()
print(f"Loaded {len(revenue_df)} quarters of revenue data")
print(revenue_df)

print("\nLoading production mix data from Google Sheets...")
prod_mix_df = pd.read_csv(PROD_MIX_URL)
# Clean up any extra columns - quarter format is "FY24 Q1"
prod_mix_df = prod_mix_df[['quarter', 'chip_type', 'share_p5', 'share_p95']].dropna()
print(f"Loaded {len(prod_mix_df)} version-quarter combinations")
print(prod_mix_df)

Loading AMD Instinct revenue data from Google Sheets...
Loaded 7 quarters of revenue data
   quarter  instinct_revenue_p5  instinct_revenue_p95
0  FY24 Q1                  0.6                  0.90
1  FY24 Q2                  1.0                  1.25
2  FY24 Q3                  1.4                  1.60
3  FY24 Q4                  1.7                  1.90
4  FY25 Q1                  1.3                  1.80
5  FY25 Q2                  0.8                  1.30
6  FY25 Q3                  1.7                  2.40

Loading production mix data from Google Sheets...
Loaded 22 version-quarter combinations
    quarter chip_type  share_p5  share_p95
0   FY24 Q1    MI300A      0.01       0.05
1   FY24 Q1    MI250X      0.25       0.50
2   FY24 Q1    MI300X      0.45       0.60
3   FY24 Q2    MI300X      0.75       0.90
4   FY24 Q2    MI300A      0.02       0.10
5   FY24 Q2    MI250X      0.01       0.10
6   FY24 Q3    MI300X      0.70       0.90
7   FY24 Q3    MI300A      0.15       0.25
8

In [19]:
# ==============================================
# CONVERT TO SQUIGGLEPY DISTRIBUTIONS
# ==============================================

# Convert revenue DataFrame to dictionary of distributions (revenue in billions)
AMD_REVENUE = {}
for _, row in revenue_df.iterrows():
    quarter = row['quarter']
    # Revenue values are in billions
    AMD_REVENUE[quarter] = sq.norm(row['instinct_revenue_p5'], row['instinct_revenue_p95'])

print(f"Created {len(AMD_REVENUE)} revenue distributions")
print(f"Quarters: {list(AMD_REVENUE.keys())}")

# Convert production mix DataFrame to nested dictionary of distributions
PROD_MIX = {}
for quarter in prod_mix_df['quarter'].unique():
    quarter_data = prod_mix_df[prod_mix_df['quarter'] == quarter]
    PROD_MIX[quarter] = {}
    for _, row in quarter_data.iterrows():
        chip_type = row['chip_type']
        PROD_MIX[quarter][chip_type] = sq.norm(row['share_p5'], row['share_p95'], lclip=0, rclip=1)

print(f"\nCreated production mix for {len(PROD_MIX)} quarters")
for q in PROD_MIX:
    print(f"  {q}: {list(PROD_MIX[q].keys())}")

Created 7 revenue distributions
Quarters: ['FY24 Q1', 'FY24 Q2', 'FY24 Q3', 'FY24 Q4', 'FY25 Q1', 'FY25 Q2', 'FY25 Q3']

Created production mix for 7 quarters
  FY24 Q1: ['MI300A', 'MI250X', 'MI300X']
  FY24 Q2: ['MI300X', 'MI300A', 'MI250X']
  FY24 Q3: ['MI300X', 'MI300A', 'MI250X']
  FY24 Q4: ['MI300X', 'MI300A', 'MI250X']
  FY25 Q1: ['MI300X', 'MI325X', 'MI300A']
  FY25 Q2: ['MI300X', 'MI325X', 'MI350X']
  FY25 Q3: ['MI300X', 'MI325X', 'MI350X', 'MI355X']


In [20]:
# ==============================================
# AMD INSTINCT SPECS AND PRICES
# ==============================================

# AMD Instinct specs: 8-bit TOPS
AMD_SPECS = {
    'MI250X': {'tops': 383},
    'MI300X': {'tops': 2615},
    'MI300A': {'tops': 1961},
    'MI325X': {'tops': 2615},
    'MI350X': {'tops': 4614},
    'MI355X': {'tops': 5033},
}

# Average Selling Prices (ASP)
# TODO: Consider changing to normal or lognormal distribution for more realistic price modeling
# Using uniform for now to be consistent with other estimates
AMD_PRICES = {
    # MI300X: Price between $10k and $15k... MSFT pays roughly $10k
    'MI300X': sq.uniform(10000, 15000),
    # MI300A: El Capitan was 44.5K MI300A and we estimate that cost $300M in compute which comes out to $6.7k per GPU
    'MI300A': sq.uniform(6000, 7500),
    # MI325X: $10K - $20K
    'MI325X': sq.uniform(10000, 20000),
    # Legacy / Other (MI250X): $8k - 12k
    'MI250X': sq.uniform(8000, 12000),
    # MI350X: $22k - $25k. lower end of $22k assuming bulk discount
    'MI350X': sq.uniform(22000, 25000),
    # MI355X
    'MI355X': sq.uniform(25000, 30000),
}

In [21]:
# ==============================================
# SAMPLING FUNCTIONS
# ==============================================

def sample_revenue(quarter):
    """Draw one sample from the quarter's revenue distribution (returns dollars)."""
    return (AMD_REVENUE[quarter] @ 1) * B

def sample_shares(quarter):
    """Draw one sample from each chip type's share distribution and normalize."""
    mix = PROD_MIX[quarter]
    raw_shares = {chip_type: dist @ 1 for chip_type, dist in mix.items()}
    return normalize_shares(raw_shares)

def sample_price(quarter, chip_type):
    """Draw one sample from the chip type's price distribution."""
    return AMD_PRICES[chip_type] @ 1

In [22]:
# ==============================================
# RUN MONTE CARLO SIMULATION
# ==============================================

sim_results = estimate_chip_sales(
    quarters=list(AMD_REVENUE.keys()),
    versions=list(AMD_SPECS.keys()),
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_price=sample_price,
    n_samples=N_SAMPLES
)

In [23]:
# ==============================================
# SUMMARIZE QUARTERLY RESULTS
# ==============================================

def summarize_results(results):
    """Create summary DataFrame with percentiles."""
    rows = []
    for quarter in results:
        row = {'Quarter': quarter}
        total = np.zeros(N_SAMPLES)
        for chip_type in AMD_SPECS:
            arr = np.array(results[quarter][chip_type])
            total += arr
            if arr.sum() > 0:
                row[f'{chip_type}_p50'] = int(np.percentile(arr, 50))
        row['total_p5'] = int(np.percentile(total, 5))
        row['total_p50'] = int(np.percentile(total, 50))
        row['total_p95'] = int(np.percentile(total, 95))
        rows.append(row)
    return pd.DataFrame(rows)

df = summarize_results(sim_results)
print("AMD Instinct Production Volumes by Quarter (chips)")
print(df[['Quarter', 'total_p5', 'total_p50', 'total_p95']].to_string(index=False))

AMD Instinct Production Volumes by Quarter (chips)
Quarter  total_p5  total_p50  total_p95
FY24 Q1     53281      68371      85556
FY24 Q2     79685      96072     117615
FY24 Q3    122068     141475     165107
FY24 Q4    147541     169204     196966
FY25 Q1     95676     121374     151114
FY25 Q2     55697      75962     100617
FY25 Q3     91333     113860     139460


In [24]:
# ==============================================
# CUMULATIVE TOTALS BY CHIP TYPE
# ==============================================
# Note: don't trust the confidence intervals here, because they don't account 
# for correlation across quarters. This means they are probably too narrow.

cumulative = {chip_type: np.zeros(N_SAMPLES) for chip_type in AMD_SPECS}
for quarter in sim_results:
    for chip_type in AMD_SPECS:
        cumulative[chip_type] += np.array(sim_results[quarter][chip_type])

print_cumulative_summary(cumulative, AMD_SPECS, "Cumulative AMD Instinct Production (Q1 2024 - Q3 2025)")


Cumulative AMD Instinct Production (Q1 2024 - Q3 2025)
Version           p5          p50          p95
---------------------------------------------
MI250X       32,867       44,778       58,326
MI300X      449,238      500,273      553,482
MI300A       95,938      114,621      133,770
MI325X       56,394       77,730      104,708
MI350X       31,291       40,301       51,251
MI355X        8,001       12,277       17,525
---------------------------------------------
TOTAL       732,473      791,585      853,547


In [25]:
# ==============================================
# H100 EQUIVALENTS
# ==============================================

h100_eq = compute_h100_equivalents(cumulative, AMD_SPECS, H100_TOPS)
print_cumulative_summary(h100_eq, AMD_SPECS, "H100 Equivalents (8-bit TOPS basis)")


H100 Equivalents (8-bit TOPS basis)
Version           p5          p50          p95
---------------------------------------------
MI250X        6,360        8,666       11,288
MI300X      593,612      661,048      731,358
MI300A       95,066      113,579      132,554
MI325X       74,518      102,710      138,358
MI350X       72,955       93,963      119,491
MI355X       20,349       31,223       44,571
---------------------------------------------
TOTAL       935,436    1,013,992    1,095,152


In [26]:
# ==============================================
# ANNUAL TOTALS
# ==============================================

def annual_totals(results):
    """Calculate annual totals from quarterly results."""
    year_totals = {'FY24': np.zeros(N_SAMPLES), 'FY25': np.zeros(N_SAMPLES)}
    for quarter in results:
        # Extract fiscal year from "FY24 Q1" format
        fy = quarter.split()[0]  # "FY24"
        for chip_type in AMD_SPECS:
            year_totals[fy] += np.array(results[quarter][chip_type])
    return year_totals

yearly = annual_totals(sim_results)
print("\nTotal AMD Instinct Production by Fiscal Year")
for year in ['FY24', 'FY25']:
    p5, p50, p95 = [int(np.percentile(yearly[year], p)) for p in [5, 50, 95]]
    print(f"{year}: {p50:,} chips (90% CI: {p5:,} - {p95:,})")


Total AMD Instinct Production by Fiscal Year
FY24: 478,057 chips (90% CI: 436,704 - 521,757)
FY25: 313,159 chips (90% CI: 270,569 - 358,118)


In [27]:
# ==============================================
# EXPORT TO CSV
# ==============================================

from datetime import datetime

# AMD uses calendar quarters, FY24 = calendar 2024
def get_quarter_dates(quarter_str):
    """Convert quarter string like 'FY24 Q1' to start and end dates."""
    parts = quarter_str.split()
    fy = int(parts[0][2:])  # FY24 -> 24
    q_num = int(parts[1][1])  # Q1 -> 1
    year = 2000 + fy  # 24 -> 2024
    
    quarter_dates = {
        1: (f"{year}-01-01", f"{year}-03-31"),
        2: (f"{year}-04-01", f"{year}-06-30"),
        3: (f"{year}-07-01", f"{year}-09-30"),
        4: (f"{year}-10-01", f"{year}-12-31"),
    }
    return quarter_dates[q_num]

# Get current timestamp for notes
timestamp = datetime.now().strftime("%m-%d-%Y %H:%M")
generated_note = f"Date estimates were generated: {timestamp}"

# Create rows for output
rows = []

for quarter in sim_results:
    start_date, end_date = get_quarter_dates(quarter)
    
    for chip_type in AMD_SPECS:
        arr = np.array(sim_results[quarter][chip_type])
        if arr.sum() > 0:
            # Calculate H100 equivalents
            h100e_arr = arr * (AMD_SPECS[chip_type]['tops'] / H100_TOPS)
            
            rows.append({
                'Name': f"{quarter} - {chip_type}",
                'Chip manufacturer': 'AMD',
                'Start date': start_date,
                'End date': end_date,
                'Compute estimate in H100e (median)': int(np.percentile(h100e_arr, 50)),
                'H100e (5th percentile)': int(np.percentile(h100e_arr, 5)),
                'H100e (95th percentile)': int(np.percentile(h100e_arr, 95)),
                'Number of Units': int(np.percentile(arr, 50)),
                'Number of Units (5th percentile)': int(np.percentile(arr, 5)),
                'Number of Units (95th percentile)': int(np.percentile(arr, 95)),
                'Source / Link': '',
                'Notes': generated_note,
                'Chip type': chip_type,
                'Last Modified By': '',
                'Last Modified': '',
            })

# Create output dataframe
amd_timelines = pd.DataFrame(rows)

# Save to CSV
output_path = 'amd_chip_timelines.csv'
amd_timelines.to_csv(output_path, index=False)

print(f"Exported to {output_path}")
print(f"\n{len(rows)} rows exported")
print(amd_timelines[['Name', 'Chip manufacturer', 'Start date', 'End date', 
                      'Compute estimate in H100e (median)', 'Number of Units', 'Chip type']].to_string())

Exported to amd_chip_timelines.csv

22 rows exported
                Name Chip manufacturer  Start date    End date  Compute estimate in H100e (median)  Number of Units Chip type
0   FY24 Q1 - MI250X               AMD  2024-01-01  2024-03-31                                5788            29909    MI250X
1   FY24 Q1 - MI300X               AMD  2024-01-01  2024-03-31                               44889            33971    MI300X
2   FY24 Q1 - MI300A               AMD  2024-01-01  2024-03-31                                3496             3528    MI300A
3   FY24 Q2 - MI250X               AMD  2024-04-01  2024-06-30                                1273             6582    MI250X
4   FY24 Q2 - MI300X               AMD  2024-04-01  2024-06-30                              103999            78705    MI300X
5   FY24 Q2 - MI300A               AMD  2024-04-01  2024-06-30                               10482            10578    MI300A
6   FY24 Q3 - MI250X               AMD  2024-07-01  2024-09-30   